# Query book

In [ ]:
import lamindb as ln
import lamindb.schema as lns

from sqlmodel import or_

ln.track()

## Basic select operations

### Output formats

In [ ]:
ln.select(ln.File).df()

In [ ]:
ln.select(ln.File).all()

In [ ]:
ln.select(ln.File).first()

In [ ]:
ln.select(ln.File, name="Mouse Lymph Node scRNA-seq", suffix=".h5ad").one()

### And

In [ ]:
ln.select(ln.File).where(
    ln.File.name == "Mouse Lymph Node scRNA-seq", ln.File.suffix == ".h5ad"
).one()

### Or

In [ ]:
ln.select(ln.File).where(or_(ln.File.suffix == ".zarr", ln.File.suffix == ".h5ad")).df()

### In

In [ ]:
ln.select(ln.File).where(ln.File.suffix.in_([".zarr", ".h5ad"])).df()

### Order by

In [ ]:
ln.select(ln.Run).order_by(ln.Run.created_at.desc()).df()

### Contains

In [ ]:
ln.select(ln.Transform).where(ln.Transform.title.contains("Link")).df()

## Data lineage

### Track run, inputs & outputs

Which run does this file `output.bam` come from?

In [ ]:
ss = ln.Session()

run = ss.select(ln.Run).join(ln.File, name="output", suffix=".bam").one()

In [ ]:
run.inputs

In [ ]:
run.outputs

In [ ]:
ss.close()

### Track notebook

Which notebook ingested this dataset `Mouse Lymph Node scRNA-seq.h5ad`?

In [ ]:
ln.select(ln.Transform).join(ln.Run).join(
    ln.File, name="Mouse Lymph Node scRNA-seq", suffix=".h5ad"
).one()

Which notebooks are produced by testuser2?

In [ ]:
ln.select(ln.Transform).join(lns.User, handle="testuser2").df()

### Track pipeline

Which pipeline produced this file `input.fastq.gz`?

In [ ]:
with ln.Session() as ss:
    file = ss.select(ln.File, name="input", suffix=".fastq.gz").one()
    print(file.source.transform)

Which pipelines are registered by testuser1?

In [ ]:
ln.select(ln.Transform).join(lns.User, handle="testuser1").df()

### Track user

Which users have interacted with the database?

In [ ]:
ln.select(lns.User).df()

Which user ingested this file `input.fastq.gz`?

In [ ]:
ln.select(lns.User).join(ln.Run).join(ln.File, name="input", suffix=".fastq.gz").one()

Which users created notebooks with "lineage" in the title?

In [ ]:
ln.select(lns.User).join(ln.Transform).where(
    ln.Transform.title.contains("lineage")
).df()

Which user registered this pipeline?

In [ ]:
ln.select(lns.User).join(ln.Transform, name="Cell Ranger", v="7").one()